In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.4.2


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Minimum Water Level', variable_name=('A.1_Water Level', 'A.2_Water Level', 'A.3_Water Level', 'A.4_Water Level', 'A.5_Water Level'), function=<function min_over_pf2 at 0x11b561d00>)
ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x11b561b20>)
ScalarOutcome('Dike Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs'), function=<function sum_over at 0x11b561b20>)
ScalarOutcome('RfR Investment Costs', variable_name=('RfR Total Costs',), function=<function sum_over at 0x11b561b20>)
ScalarOutcome('Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x11b561b20>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_

In [8]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 14 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:02<00:00, 87.52it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [9]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['Minimum Water Level', 'Expected Annual Damage', 'Dike Investment Costs', 'RfR Investment Costs', 'Evacuation Costs', 'Expected Number of Deaths'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,85,305.580230,10.0,0.275834,92.561750,1.0,0.679593,69.629725,10.0,0.074087,...,10,3,7,3,9,3,1,4,0,dikesnet
1,16,183.280907,1.5,0.290083,42.402895,10.0,0.131683,285.643580,10.0,0.469751,...,10,3,7,3,9,3,1,5,0,dikesnet
2,72,188.233441,10.0,0.583373,202.318302,10.0,0.246604,242.976369,1.5,0.244254,...,10,3,7,3,9,3,1,6,0,dikesnet
3,49,42.776954,10.0,0.605212,227.811525,10.0,0.115273,102.616454,1.5,0.122505,...,10,3,7,3,9,3,1,7,0,dikesnet
4,70,197.021204,1.0,0.986181,237.097982,10.0,0.054958,289.680635,10.0,0.495921,...,10,3,7,3,9,3,1,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,119,258.498579,1.5,0.214939,285.379032,1.5,0.593914,220.612892,1.0,0.875599,...,6,8,5,8,1,8,0,49,3,dikesnet
196,11,274.783545,1.5,0.498455,103.521866,10.0,0.270396,160.168068,1.5,0.210188,...,6,8,5,8,1,8,0,50,3,dikesnet
197,101,293.502867,1.0,0.960667,123.452877,1.0,0.850001,176.774326,1.0,0.014859,...,6,8,5,8,1,8,0,51,3,dikesnet
198,15,158.290824,10.0,0.711779,205.875791,1.0,0.802672,327.567383,1.0,0.048961,...,6,8,5,8,1,8,0,52,3,dikesnet


In [10]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,Minimum Water Level,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths
0,4.298700,2.177121e+07,6.079357e+08,8.137000e+08,369.415316,0.010508
1,4.618712,1.138555e+07,6.079357e+08,8.137000e+08,424.320510,0.004586
2,4.298700,1.889399e+06,6.079357e+08,8.137000e+08,30.428777,0.000918
3,4.685625,5.264243e+06,6.079357e+08,8.137000e+08,140.328051,0.004098
4,4.298700,4.657608e+06,6.079357e+08,8.137000e+08,91.352143,0.001221
...,...,...,...,...,...,...
195,5.228382,5.461748e+06,7.255855e+08,1.087700e+09,0.000000,0.006467
196,5.862888,1.279511e+07,7.255855e+08,1.087700e+09,0.000000,0.017777
197,4.731774,1.769701e+07,7.255855e+08,1.087700e+09,0.000000,0.023207
198,5.219743,2.944012e+06,7.255855e+08,1.087700e+09,0.000000,0.003376


In [11]:
from ema_workbench import save_results
save_results(results,'dike_experiments_2.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/amaryllisbrosens/PycharmProjects/MBDM-Group-1/dike_experiments_2.tar.gz


In [12]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [13]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 14 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [00:03<00:00, 84.82it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [14]:
experiments, outcomes = results

In [15]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,Minimum Water Level,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths
0,4.538700,1.115594e+08,5.397251e+07,253800000.0,0.0,0.101991
1,4.898161,1.147167e+09,5.397251e+07,253800000.0,0.0,1.834322
2,4.538700,2.458109e+08,5.397251e+07,253800000.0,0.0,0.271896
3,4.538700,5.351370e+08,5.397251e+07,253800000.0,0.0,0.643472
4,4.791134,9.227222e+08,5.397251e+07,253800000.0,0.0,1.894281
...,...,...,...,...,...,...
295,4.638040,2.295720e+09,2.879840e+07,369700000.0,0.0,1.246198
296,4.953953,2.490523e+08,2.879840e+07,369700000.0,0.0,0.161621
297,4.418700,1.975385e+08,2.879840e+07,369700000.0,0.0,0.212585
298,5.349045,1.415553e+08,2.879840e+07,369700000.0,0.0,0.145352


In [16]:
# save results
from ema_workbench import save_results
save_results(results,'dike_experiments.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/amaryllisbrosens/PycharmProjects/MBDM-Group-1/dike_experiments.tar.gz


In [17]:
# Amaryllis: Trying to make some own policies

# def get_do_nothing_dict():
#     return {l.name: 0 for l in dike_model.levers}
#
#
# #RFR in Doesburg and Zuthpen as they talked about in the debate
# policy_rfr_A1_A3 = Policy(
#         "policy RfR in A1 and A3",
#         **dict(
#             get_do_nothing_dict(),
#             **{"1_RfR 0": 1, "1_RfR 1": 1, "1_RfR 2": 1,
#                "3_RfR 0": 1, "3_RfR 1": 1, '3_RfR 2': 1,}
#         )
#     )
#
# #RFR in Doesburg and Zuthpen and dike heightening in Cortenoever and Gorssel
# policy_rfr_A1_A3_dike_A2_A4= Policy(
#         "policy RfR in A1 and A3 and Dike height in A2 and A4",
#         **dict(
#             get_do_nothing_dict(),
#             **{# RfR for A1
#             "1_RfR 0": 1, "1_RfR 1": 1, "1_RfR 2": 1,
#             # RfR for A3
#             "3_RfR 0": 1, "3_RfR 1": 1, "3_RfR 2": 1,
#             # DikeIncrease for A2
#            # "2_DikeIncrease 0": 5, "2_DikeIncrease 1": 5, "2_DikeIncrease 2": 5,
#             # DikeIncrease for A4
#             "4_DikeIncrease 0": 5, "4_DikeIncrease 1": 5, "4_DikeIncrease 2": 5,}
#         )
#     )

#some random policies
import random
from ema_workbench import Policy

# def generate_random_policy(name):
#     lever_values = {}
#     for lever in dike_model.levers:
#         if "RfR" in lever.name:
#             # RfR levers: either 0 or 1
#             lever_values[lever.name] = random.choice([0, 1])
#         elif "DikeIncrease" in lever.name:
#             # DikeIncrease levers: 0 to 10
#             lever_values[lever.name] = random.randint(0, 10)
#         else:
#             # other levers if any - set to 0 or default
#             lever_values[lever.name] = 0
#     return Policy(name, **lever_values)
#
# Create policies 4 to 10
# random_policies = [generate_random_policy(f"random policy {i}") for i in range(4, 11)]
#
#
# policies = [
#     Policy("do nothing", **get_do_nothing_dict()),
#     policy_rfr_A1_A3,
#     policy_rfr_A1_A3_dike_A2_A4,
# ] + random_policies


In [21]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 1000
policies = 20
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 14 workers
[MainProcess/INFO] performing 1000 scenarios * 20 policies * 1 model(s) = 20000 experiments
100%|████████████████████████████████████| 20000/20000 [04:10<00:00, 79.78it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [22]:
experiments, outcomes = results
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,Minimum Water Level,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths
0,4.298700,0.000000e+00,7.515352e+08,7.701000e+08,0.000000,0.000000
1,4.487376,0.000000e+00,7.515352e+08,7.701000e+08,0.000000,0.000000
2,4.298700,0.000000e+00,7.515352e+08,7.701000e+08,0.000000,0.000000
3,4.685625,0.000000e+00,7.515352e+08,7.701000e+08,0.000000,0.000000
4,4.298700,0.000000e+00,7.515352e+08,7.701000e+08,0.000000,0.000000
...,...,...,...,...,...,...
19995,4.298700,6.669389e+06,5.967723e+08,1.274800e+09,246.920637,0.002708
19996,4.432520,0.000000e+00,5.967723e+08,1.274800e+09,0.000000,0.000000
19997,4.298700,6.407074e+06,5.967723e+08,1.274800e+09,237.744626,0.002594
19998,4.566721,1.064131e+07,5.967723e+08,1.274800e+09,242.264476,0.002650


In [23]:
# save results
from ema_workbench import save_results
save_results(results,'dike_experiments_3.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/amaryllisbrosens/PycharmProjects/MBDM-Group-1/dike_experiments_3.tar.gz
